In [1]:
from importlib import reload
# import VAE as VAE
import VAE.DataLoader as DataLoader
import VAE.Encoder as Encoder
import VAE.Decoder as Decoder
import VAE.ConvSeq2Seq as ConvSeq2Seq
import VAE.Discriminator as Discriminator
import util as util
import torch
from torch import nn
import torch.optim as optim
import math
from functools import partial

In [6]:
reload(Encoder)
reload(Decoder)
reload(ConvSeq2Seq)
reload(Discriminator)
reload(DataLoader)
reload(util)

<module 'util' from '/home/wichen/repos/CS7643_DL/Project/util.py'>

In [2]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


In [4]:
dloader = DataLoader.DataLoader(50, 25, './data/h3.6m/dataset')

Reading subject 1, action walking, subaction 1
Reading subject 1, action walking, subaction 2
Reading subject 1, action eating, subaction 1
Reading subject 1, action eating, subaction 2
Reading subject 1, action smoking, subaction 1
Reading subject 1, action smoking, subaction 2
Reading subject 1, action discussion, subaction 1
Reading subject 1, action discussion, subaction 2
Reading subject 1, action directions, subaction 1
Reading subject 1, action directions, subaction 2
Reading subject 1, action greeting, subaction 1
Reading subject 1, action greeting, subaction 2
Reading subject 1, action phoning, subaction 1
Reading subject 1, action phoning, subaction 2
Reading subject 1, action posing, subaction 1
Reading subject 1, action posing, subaction 2
Reading subject 1, action purchases, subaction 1
Reading subject 1, action purchases, subaction 2
Reading subject 1, action sitting, subaction 1
Reading subject 1, action sitting, subaction 2
Reading subject 1, action sittingdown, subacti

In [5]:
print('Memory usage before construct network: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))

Memory usage before construct network: 0 bytes => 0.0 GB


In [6]:
lt_encoder = Encoder.Encoder(16,enc_shape=[None, 49, 54, 1], enc_dim_desc={ 'hidden_num': 512,'class_num': 15 }, stride=(2,2))

st_encoder = Encoder.Encoder(16,enc_shape=[None, 20, 54, 1], enc_dim_desc={ 'hidden_num': 512}, stride=(2,2))
decoder = Decoder.Decoder(st_encoder)

generator = ConvSeq2Seq.ConvSeq2Seq(lt_encoder, decoder, window_length=20, device=device)

d_encoder = Encoder.Encoder(32, enc_shape=[None, 75,54,1], enc_dim_desc={ 'hidden_num': 512}, stride=(2,2))
discriminator = Discriminator.Discriminator(32, d_encoder).to(device)

print('Memory usage after construct network: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
                                                   torch.cuda.memory_allocated(device=device)/1e9))

Memory usage after construct network: 67533312 bytes => 0.067533312 GB


In [7]:
for name, param in generator.named_parameters():
    if param.requires_grad:
        print(name, param.numel())

encoder.conv1.weight 896
encoder.batch_norm1.weight 64
encoder.batch_norm1.bias 64
encoder.conv2.weight 114688
encoder.batch_norm2.weight 128
encoder.batch_norm2.bias 128
encoder.conv3.weight 229376
encoder.batch_norm3.weight 128
encoder.batch_norm3.bias 128
encoder.fc.weight 3305344
encoder.fc.bias 527
decoder.st_encoder.conv1.weight 896
decoder.st_encoder.batch_norm1.weight 64
decoder.st_encoder.batch_norm1.bias 64
decoder.st_encoder.conv2.weight 114688
decoder.st_encoder.batch_norm2.weight 128
decoder.st_encoder.batch_norm2.bias 128
decoder.st_encoder.conv3.weight 229376
decoder.st_encoder.batch_norm3.weight 128
decoder.st_encoder.batch_norm3.bias 128
decoder.st_encoder.fc.weight 1376256
decoder.st_encoder.fc.bias 512
decoder.fc1.weight 524288
decoder.fc1.bias 512
decoder.fc2.weight 27648
decoder.fc2.bias 54


In [8]:
for name, param in discriminator.named_parameters():
    if param.requires_grad:
        print(name, param.numel())

d_encoder.conv1.weight 1792
d_encoder.batch_norm1.weight 128
d_encoder.batch_norm1.bias 128
d_encoder.conv2.weight 458752
d_encoder.batch_norm2.weight 256
d_encoder.batch_norm2.bias 256
d_encoder.conv3.weight 917504
d_encoder.batch_norm3.weight 256
d_encoder.batch_norm3.bias 256
d_encoder.fc.weight 9175040
d_encoder.fc.bias 512
fc1.weight 134912
fc1.bias 256
fc2.weight 256
fc2.bias 1


In [ ]:
    lt_encoder = Encoder.Encoder(16,enc_shape=[None, 49, 54, 1], 
                                 enc_dim_desc={ 'hidden_num': 512,'class_num': 15 }, 
                                 stride=(2,2), 
                                 kernel_size=(2, 7))

    st_encoder = Encoder.Encoder(16,enc_shape=[None, 20, 54, 1], 
                                 enc_dim_desc={ 'hidden_num': 512}, 
                                 stride=(2,2), 
                                 kernel_size=(2, 7))
    decoder = Decoder.Decoder(st_encoder)

    generator = ConvSeq2Seq.ConvSeq2Seq(lt_encoder, decoder, window_length=20, device=device)

    d_encoder = Encoder.Encoder(32, enc_shape=[None, 75,54,1], 
                                enc_dim_desc={ 'hidden_num': 512}, 
                                stride=(2,2), 
                                kernel_size=(2, 7))
    discriminator = Discriminator.Discriminator(32, d_encoder).to(device)

## Training

In [8]:
util.train(dloader, 
           generator, 
           discriminator, 
           device=device,
           batch = 16,
           lr=5e-5,
           lr_decay_steps = 10000,
           lr_decay = 0.99,
           L2_lambda = 0.001,
           iterations = 1000,
           display = 100,
           tensorboard=True,
           save_model=True,
           model_name='default_setting_1000_itr')

Iterations 0 loss_d 1.387010, loss_g 0.276363, lr 0.000050, time 0.867337
Iterations 100 loss_d 1.374417, loss_g 0.248493, lr 0.000050, time 64.358381
Iterations 200 loss_d 1.255111, loss_g 0.232910, lr 0.000050, time 126.736285
Iterations 300 loss_d 1.155149, loss_g 0.227703, lr 0.000050, time 188.517448
Iterations 400 loss_d 1.085928, loss_g 0.209014, lr 0.000050, time 249.691169
Iterations 500 loss_d 1.100226, loss_g 0.224744, lr 0.000050, time 308.562355
Iterations 600 loss_d 1.012678, loss_g 0.213151, lr 0.000050, time 370.601089
Iterations 700 loss_d 1.015243, loss_g 0.200227, lr 0.000050, time 433.138354
Iterations 800 loss_d 0.983804, loss_g 0.209434, lr 0.000050, time 496.688075
Iterations 900 loss_d 0.987851, loss_g 0.212384, lr 0.000050, time 561.508905
Iterations 1000 loss_d 0.948644, loss_g 0.206390, lr 0.000050, time 624.843547


KeyboardInterrupt: 

## Hyperparameter Tuning

In [3]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from hyperopt.pyll.base import scope 
import pickle

In [4]:
dloader = DataLoader.DataLoader(50, 25, './data/h3.6m/dataset')

Reading subject 1, action walking, subaction 1
Reading subject 1, action walking, subaction 2
Reading subject 1, action eating, subaction 1
Reading subject 1, action eating, subaction 2
Reading subject 1, action smoking, subaction 1
Reading subject 1, action smoking, subaction 2
Reading subject 1, action discussion, subaction 1
Reading subject 1, action discussion, subaction 2
Reading subject 1, action directions, subaction 1
Reading subject 1, action directions, subaction 2
Reading subject 1, action greeting, subaction 1
Reading subject 1, action greeting, subaction 2
Reading subject 1, action phoning, subaction 1
Reading subject 1, action phoning, subaction 2
Reading subject 1, action posing, subaction 1
Reading subject 1, action posing, subaction 2
Reading subject 1, action purchases, subaction 1
Reading subject 1, action purchases, subaction 2
Reading subject 1, action sitting, subaction 1
Reading subject 1, action sitting, subaction 2
Reading subject 1, action sittingdown, subacti

In [7]:
space = {'lr': 5e-5,
         'L2_lambda': 0.001,
         'lt_encoder_filters': 16,
         'st_encoder_filters': 16,
         'd_encoder_filters': 32,
         'discriminator_output_filters': 32,
         'kernel_height': scope.int(hp.quniform('kernel_height',2,10,1)),
         'kernel_width': scope.int(hp.quniform('kernel_width',5,20,2)),
         'stride_vert': 2,
         'stride_hori': 2,
         'device': device
}

# space = {'lr': hp.uniform('lr', 0.000001, 0.01),
#          'L2_lambda': hp.uniform('L2_lambda', 0.001, 0.1),
#          'lt_encoder_filters': scope.int(hp.quniform('lt_encoder_filters',4,64,4)),
#          'st_encoder_filters': scope.int(hp.quniform('st_encoder_filters',4,64,4)),
#          'd_encoder_filters': scope.int(hp.quniform('d_encoder_filters',4,64,4)),
#          'discriminator_output_filters': scope.int(hp.quniform('discriminator_output_filters',4,64,4)),
#          'kernel_height': scope.int(hp.quniform('kernel_height',2,10,1)),
#          'kernel_width': scope.int(hp.quniform('kernel_width',5,20,2)),
#          'stride_vert': scope.int(hp.quniform('stride_vert',1,5,1)),
#          'stride_hori': scope.int(hp.quniform('stride_hori',2,5,1)),
#          'device': device
# }

trials = Trials()
best = fmin(partial(util.optimize, dloader), 
            space, 
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
pickle.dump(trials, open("{}/trial_{}.pkl".format('tuning_save_dir', 0), 'wb'))

Using device: cuda
Training on following parameters: 
lr=0.00005000
L2_lambda=0.00
lt_encoder_filters size=16.00
st_encoder_filters size=16.00
d_encoder_filters size=32.00
discriminator_output_filters size=32.00
kernel size=(9,6)
stride=(2,2)
Iterations 0 loss_d 1.387902, loss_g 0.273694, lr 0.000050, time 2.554062
Iterations 100 loss_d 1.358522, loss_g 0.224902, lr 0.000050, time 252.704688
Iterations 200 loss_d 1.097444, loss_g 0.229988, lr 0.000050, time 508.124125
Iterations 300 loss_d 0.829321, loss_g 0.230110, lr 0.000050, time 777.423306
Iterations 400 loss_d 0.629069, loss_g 0.222812, lr 0.000050, time 1048.998810
Iterations 500 loss_d 0.596275, loss_g 0.232395, lr 0.000050, time 1322.922085
Iterations 600 loss_d 0.512243, loss_g 0.226099, lr 0.000050, time 1581.479376
Iterations 700 loss_d 0.510012, loss_g 0.218592, lr 0.000050, time 1823.992167
Iterations 800 loss_d 0.456715, loss_g 0.213946, lr 0.000050, time 2044.261043
Iterations 900 loss_d 0.520800, loss_g 0.223337, lr 0.

## Inference

In [11]:
dloader = DataLoader.DataLoader(50, 25, './data/h3.6m/dataset')

Reading subject 1, action walking, subaction 1
Reading subject 1, action walking, subaction 2
Reading subject 1, action eating, subaction 1
Reading subject 1, action eating, subaction 2
Reading subject 1, action smoking, subaction 1
Reading subject 1, action smoking, subaction 2
Reading subject 1, action discussion, subaction 1
Reading subject 1, action discussion, subaction 2
Reading subject 1, action directions, subaction 1
Reading subject 1, action directions, subaction 2
Reading subject 1, action greeting, subaction 1
Reading subject 1, action greeting, subaction 2
Reading subject 1, action phoning, subaction 1
Reading subject 1, action phoning, subaction 2
Reading subject 1, action posing, subaction 1
Reading subject 1, action posing, subaction 2
Reading subject 1, action purchases, subaction 1
Reading subject 1, action purchases, subaction 2
Reading subject 1, action sitting, subaction 1
Reading subject 1, action sitting, subaction 2
Reading subject 1, action sittingdown, subacti

In [9]:
util.InferenceSample(dloader, generator, model_name='default_setting_1000_itr', )


walking

milliseconds     |    80 |   160 |   320 |   400 |   560 |  1000 |
walking          | 0.323 | 0.509 | 0.638 | 0.669 | 0.658 | 0.630 |

eating

milliseconds     |    80 |   160 |   320 |   400 |   560 |  1000 |
eating           | 0.232 | 0.354 | 0.497 | 0.586 | 0.641 | 0.805 |

smoking

milliseconds     |    80 |   160 |   320 |   400 |   560 |  1000 |
smoking          | 0.245 | 0.424 | 0.766 | 0.704 | 0.668 | 1.038 |

discussion

milliseconds     |    80 |   160 |   320 |   400 |   560 |  1000 |
discussion       | 0.282 | 0.572 | 0.707 | 0.749 | 1.066 | 1.119 |

directions

milliseconds     |    80 |   160 |   320 |   400 |   560 |  1000 |
directions       | 0.384 | 0.542 | 0.638 | 0.680 | 0.685 | 0.818 |

greeting

milliseconds     |    80 |   160 |   320 |   400 |   560 |  1000 |
greeting         | 0.506 | 0.776 | 1.026 | 1.121 | 1.230 | 1.047 |

phoning

milliseconds     |    80 |   160 |   320 |   400 |   560 |  1000 |
phoning          | 0.594 | 1.077 | 1.334 | 1.414 | 1.

## Grave Yard **********************************************************************************

In [12]:
iterations = 20000
for i in range(iterations):
    generator.train()
    discriminator.train()
    # print('Memory usage when training begins: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    
    # print("****************train discriminator*****************")
    encoder_data, discriminator_data, yhat = dloader.get_train_batch(16)

    encoder_data = torch.from_numpy(encoder_data).float().to(device)
    # print("encoder_data size: {} bytes => {} GB".format(encoder_data.element_size() * encoder_data.nelement(),
    #                                                     encoder_data.element_size() * encoder_data.nelement()/1e9))
    
    discriminator_data = torch.from_numpy(discriminator_data).float().to(device)
    # print("discriminator_data size: {} bytes => {} GB".format(discriminator_data.element_size() * discriminator_data.nelement(),
    #                                                           discriminator_data.element_size() * discriminator_data.nelement()/1e9))
    
    yhat = torch.from_numpy(yhat).float().to(device)
    # print("yhat size: {} bytes => {} GB".format(yhat.element_size() * yhat.nelement(),
    #                                             yhat.element_size() * yhat.nelement()/1e9))
    
    # expected_seq = discriminator_data[:, 50:, :, :]
    # print("expected_seq size: {} bytes => {} GB".format(expected_seq.element_size() * expected_seq.nelement(),
    #                                                     expected_seq.element_size() * expected_seq.nelement()/1e9))
    for param in discriminator.parameters():
        param.grad = None
    # optimizerD.zero_grad()
    # print('Memory usage after discriminator zero_grad: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    # with torch.cuda.amp.autocast():
    d_logits_real = discriminator.forward(discriminator_data, yhat)
    # print('Memory usage after d_logits_real forward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))

    d_loss_real = D_criterion(d_logits_real, torch.ones_like(d_logits_real))
    # print('Memory usage after d_loss_real: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))

    d_loss_real.backward()
    # print('Memory usage after d_loss_real backward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    # with torch.cuda.amp.autocast():
    predicted_seq, predicted_action, generated_sample = generator.forward(encoder_data, discriminator_data)
    
    # print('Memory usage after generator forward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    d_logits_fake = discriminator.forward(generated_sample, yhat)
    # print('Memory usage after d_logits_fake forward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))

    d_loss_fake = D_criterion(d_logits_fake, torch.zeros_like(d_logits_fake))
    # print('Memory usage after d_loss_fake : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))

    d_loss_fake.backward()
    # print('Memory usage after d_loss_fake backward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))

    loss_discriminator = d_loss_real + d_loss_fake
    # print('Memory usage after loss_discriminator summed: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    optimizerD.step()
    # print('Memory usage after optimizerD stepped: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    
    # print("****************train generator*****************")
    encoder_data, discriminator_data, yhat = dloader.get_train_batch(16)

    encoder_data = torch.from_numpy(encoder_data).float().to(device)
    # print("encoder_data size: {} bytes => {} GB".format(encoder_data.element_size() * encoder_data.nelement(),
    #                                                     encoder_data.element_size() * encoder_data.nelement()/1e9))
    
    discriminator_data = torch.from_numpy(discriminator_data).float().to(device)
    # print("discriminator_data size: {} bytes => {} GB".format(discriminator_data.element_size() * discriminator_data.nelement(),
    #                                                           discriminator_data.element_size() * discriminator_data.nelement()/1e9))
    
    yhat = torch.from_numpy(yhat).float().to(device)
    # print("yhat size: {} bytes => {} GB".format(yhat.element_size() * yhat.nelement(),
    #                                             yhat.element_size() * yhat.nelement()/1e9))
    
    expected_seq = discriminator_data[:, 50:, :, :]
    # print("expected_seq size: {} bytes => {} GB".format(expected_seq.element_size() * expected_seq.nelement(),
    #                                                     expected_seq.element_size() * expected_seq.nelement()/1e9))
    
    for param in generator.parameters():
        param.grad = None
    # optimizerG.zero_grad()
    # with torch.cuda.amp.autocast():
    predicted_seq, predicted_action, generated_sample = generator.forward(encoder_data, discriminator_data)
    # print('Memory usage after generator forward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    ReconstructError = G_criterion(predicted_seq, expected_seq)
    # print('Memory usage after ReconstructError : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    ReconstructError.backward(retain_graph=True)
    # print('Memory usage after ReconstructError backward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    # with torch.cuda.amp.autocast():
    d_logits_fake = discriminator.forward(generated_sample, yhat)
    # print('Memory usage after discriminator foward : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    g_loss = D_criterion(d_logits_fake, torch.ones_like(d_logits_fake)) * torch.tensor(0.01)
    # print('Memory usage after g_loss : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    g_loss.backward()
    # print('Memory usage after g_loss backward: {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    loss_generator = ReconstructError + g_loss
    # print('Memory usage after loss_generator summed : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    
    optimizerG.step()
    # print('Memory usage after optimizerG stepped : {} bytes => {} GB'.format(torch.cuda.memory_allocated(device=device), 
    #                                                torch.cuda.memory_allocated(device=device)/1e9))
    
    # Output training stats
    if i % 100 == 0:
        print('[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f'
            % (i, iterations, loss_discriminator.item(), loss_generator.item()))
    torch.cuda.empty_cache()

[0/20000]	Loss_D: 1.3884	Loss_G: 0.2619
[100/20000]	Loss_D: 1.2983	Loss_G: 0.2432
[200/20000]	Loss_D: 1.1957	Loss_G: 0.2439
[300/20000]	Loss_D: 0.9976	Loss_G: 0.3253
[400/20000]	Loss_D: 1.1649	Loss_G: 0.1919
[500/20000]	Loss_D: 1.1214	Loss_G: 0.2258
[600/20000]	Loss_D: 1.0522	Loss_G: 0.3107
[700/20000]	Loss_D: 0.9603	Loss_G: 0.2668
[800/20000]	Loss_D: 1.2217	Loss_G: 0.2764
[900/20000]	Loss_D: 0.9751	Loss_G: 0.4168
[1000/20000]	Loss_D: 0.9488	Loss_G: 0.2173
[1100/20000]	Loss_D: 1.0963	Loss_G: 0.2862
[1200/20000]	Loss_D: 0.8042	Loss_G: 0.1657
[1300/20000]	Loss_D: 0.7598	Loss_G: 0.2153
[1400/20000]	Loss_D: 0.7531	Loss_G: 0.3976
[1500/20000]	Loss_D: 1.0152	Loss_G: 0.2257
[1600/20000]	Loss_D: 0.7739	Loss_G: 0.2131
[1700/20000]	Loss_D: 0.7638	Loss_G: 0.2901
[1800/20000]	Loss_D: 0.9798	Loss_G: 0.1239
[1900/20000]	Loss_D: 1.3205	Loss_G: 0.1534
[2000/20000]	Loss_D: 0.7119	Loss_G: 0.1554
[2100/20000]	Loss_D: 0.9848	Loss_G: 0.1804
[2200/20000]	Loss_D: 0.8662	Loss_G: 0.1224
[2300/20000]	Loss_D: 0.

In [15]:
torch.save(generator.state_dict(), 'save_dir/generator.pt')

In [16]:
torch.save(discriminator.state_dict(), 'save_dir/discriminator.pt')